## Comments

### Speed
Too slow. Takes 2 seconds per 1000 on my computer, 5 seconds on Kaggle. The dataset has 1.3m questions.
Not feasible to use this method on QIQC unless this is overcome.

### Sentence reconstruction
How to recombine sentences after removing the context?
Currently I concatenate them with spaces in between, and this causes unnessary spaces before and after punctuation. 

### Context removal
Could be done better
- usually does not recognise non-proper noun NORPs

In [1]:
import numpy as np
import pandas as pd
import os
print(os.listdir("../input"))

pd.set_option('display.max_colwidth', -1)
pd.option_context("display.max_rows", 1000)

['embeddings', '.DS_Store', 'test.csv', 'train.csv', 'sample_submission.csv']


In [2]:
df = pd.read_csv("../input/train.csv")[:1000]

In [7]:
entity_map = {"PERSON": "person",
              "NORP": "liberals",
              "FAC": "building",
              "ORG": "company",
              "GPE": "country",
              "LOC": "location",
              "PRODUCT": "item",
              "EVENT": "event",
              "WORK_OF_ART": "art",
              "LAW": "Roe v. Wade",
              "LANGUAGE": "English",
              "DATE": "today",
              "TIME": "now",
              "PERCENT": "100%",
              "MONEY": "$1",
              "QUANTITY": "1",
              "ORDINAL": "first",
              "CARDINAL": "one"}

In [11]:
import spacy as sp
import time
# spacy.prefer_gpu()
start_time = time.time()
nlp = sp.load('en_core_web_sm')

start_time = time.time()
statements_new = []
docs = df['question_text'].tolist()

# https://stackoverflow.com/questions/48199353/how-to-use-spacy-in-large-dataset-with-short-sentences-efficiently
for doc in nlp.pipe(docs):
    new = []
    for i,token in enumerate(doc):
        if token.ent_iob_ == "I":
            continue
        if token.ent_iob_ == "O":
            new.append(token.text)
            continue
        new.append(entity_map[token.ent_type_])
    statement_new = " ".join(new)
    statements_new.append(statement_new)

print(time.time() - start_time)
df['question_text'] = statements_new

2.1682751178741455


In [12]:
df.loc[df['target'] == 0].sample(n=10)

,qid,question_text,target
670,001f456432715e83b43f,"If I throw an ice cube into the ocean , what is the chance that I will come in contact with one of those water molecules ( from the ice ) again in my lifetime ?",0
44,0001dffd4f210f6beedb,What do I need to know about buying a car in country as an liberals ?,0
630,001da5913d2e9a57acfa,"What role did person develop as the fictional character "" person "" ?",0
698,0020853e782d27d173fc,Why is it ironic about the meaning of salem where so many people were hanged in the crucible ?,0
767,002422f6f10c9b7c6dc0,What would you name a cognitive automation solution ?,0
240,000b5c5c4efc832a5bf6,How do you feel about relationships with people who are disabled ?,0
976,002ffebf02534b00b670,Are company descendants of person ?,0
919,002c4d82ad6ac30768ed,How does a business - major student incorporate an international dimension to his / her undergraduate education ?,0
969,002fac6e44e4b3506921,Should nt there be more women in country ?,0
735,002284840b558e065b03,"What are those kid gang genre movies like The company , company or company called ?",0


In [13]:
df.loc[df['target'] == 1].sample(n=10)

,qid,question_text,target
999,0031884048d46eea1e7f,Who will win in video game battle : Adam d angelo vs person kardasian ?,1
617,001ceedd9fd5145f89bf,"When will the country stop supporting country terrorists ( liberals ) , who are actually the same as ISIS ?",1
569,001b3c2ff060eee74cba,Why is Jinping the biggest serial religion murderer and rapist leader of the world after person ?,1
855,00290136e6ddf1846888,What 's your penis size ? Is n't that funny ?,1
289,000db4d0223af5dd9c5d,Why do all the people who claim country has great weather go silent every time there 's a new hurricane ?,1
798,00260725bbeec41eaea7,Who is paying for the royal family besides the liberals ?,1
627,001d874ba189c5e5e43e,Why do liberals need the second amendment ? Do n't want to start a fight but the right says it is necessary for a democracy and such and why are they so worried about the government .,1
224,000a898565e80fe124bf,Why are liberal minorities so voilent towards poeple with diffrent poltical beleifs ? Should supporting trump be a sentence to be imprisoned or savegely attacked ?,1
302,000e67648fce55f011be,Why do the liberals who run schools choose not to have controlled access ? The kids in country were killed due to an unlocked door .,1
144,00068875d7c82a5bcf88,"Why do liberals say they 're the superior race , when in fact it took them today until today to surpass country 's largest economy ?",1
